<a href="https://colab.research.google.com/github/akshan-main/tc/blob/main/tc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell: environment repair
!pip -q uninstall -y torchvision torchaudio
# re-assert the core stack you’re using
!pip -q install --upgrade pip
!pip -q install "torch==2.4.0" --index-url https://download.pytorch.org/whl/cu121
!pip -q install "bitsandbytes==0.43.3" "triton==3.0.0"
!pip -q install "transformers==4.43.3" "accelerate==0.34.2" "peft==0.12.0"
!pip -q install "datasets==2.20.0" "evaluate==0.4.2"
!pip -q install "scikit-learn==1.5.2" "pandas==2.2.2" "pyarrow==19.0.0"

import sys, torch, importlib
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
for m in ["evaluate","datasets","transformers","peft","accelerate","bitsandbytes","triton"]:
    try:
        importlib.import_module(m)
        print(m, "OK")
    except Exception as e:
        print(m, "ERR:", e)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires torchvision, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 1.0.22 requires torchvision, which is not installed.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompa

In [ ]:
import torch, bitsandbytes as bnb
print("bnb:", bnb.__version__)
print("GPU:", torch.cuda.get_device_name(0))

lin = bnb.nn.Linear4bit(16, 8, quant_type="nf4").to("cuda")
x = torch.randn(2, 16, device="cuda", dtype=torch.bfloat16)
y = lin(x)
print("Linear4bit OK on:", y.device)


bnb: 0.43.3
GPU: NVIDIA A100-SXM4-80GB
Linear4bit OK on: cuda:0


In [ ]:
from datasets import load_dataset
ds = load_dataset("coastalcph/lex_glue","ledgar")

label_names = ds["train"].features["label"].names
id2name = {i:n for i,n in enumerate(label_names)}
name2id = {n:i for i,n in id2name.items()}
num_labels = len(label_names)

# risky families: missing these is a false green-light
RISKY = {
    "Arbitration","Waiver Of Jury Trials","Waivers",
    "Jurisdictions","Submission To Jurisdiction","Consent To Jurisdiction","Governing Laws",
    "Modifications","Amendments",
    "Limitation Of Liability","Remedies","Indemnity","Indemnifications"
} & set(label_names)

len(label_names), sorted(list(RISKY))[:10]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

(100,
 ['Amendments',
  'Arbitration',
  'Consent To Jurisdiction',
  'Governing Laws',
  'Indemnifications',
  'Indemnity',
  'Jurisdictions',
  'Modifications',
  'Remedies',
  'Submission To Jurisdiction'])

In [ ]:
import numpy as np, pandas as pd

train_txt = ds["train"]["text"]
val_txt   = ds["validation"]["text"]
test_txt  = ds["test"]["text"]

def count_tokens_rough(texts):
    # fast, tokenizer-free proxy
    return np.array([len(t.split()) for t in texts])

VAL_TOK  = count_tokens_rough(val_txt)
VAL_CHAR = np.array([len(t) for t in val_txt])
VAL_CPT  = VAL_CHAR / np.maximum(VAL_TOK, 1)

def quartile_bucket(x, prefix):
    q1,q2,q3 = np.quantile(x, [0.25,0.5,0.75])
    def b(v):
        if v<=q1: return f"{prefix}_Q1"
        if v<=q2: return f"{prefix}_Q2"
        if v<=q3: return f"{prefix}_Q3"
        return f"{prefix}_Q4"
    return np.array([b(v) for v in x])

VAL_NTOK_BUCKET = quartile_bucket(VAL_TOK, "n_tok")
VAL_NCHR_BUCKET = quartile_bucket(VAL_CHAR, "n_char")
VAL_CPT_BUCKET  = quartile_bucket(VAL_CPT, "chars_per_tok")

FLAGS = {
    "flag_binding_arbitration": r"\bbinding arbitration\b|\barbitration\b",
    "flag_limitation_liability": r"\blimitation of liability\b|\bdisclaim(?:er|s)\b",
    "flag_governing_law": r"\bgoverning law\b|\bapplicable law\b",
    "flag_venue_jurisdiction": r"\bvenue\b|\bjurisdiction\b|\bexclusive jurisdiction\b",
    "flag_sole_discretion": r"\bsole discretion\b",
    "flag_max_extent_law": r"\bmaximum extent permitted by law\b",
    "flag_including_not_limited": r"\bincluding(?:,|\s) but not limited to\b|\bincluding without limitation\b",
}
flag_df = pd.DataFrame({
    k: pd.Series(val_txt).str.lower().str.contains(rx, regex=True, na=False).astype(int)
    for k, rx in FLAGS.items()
})
flag_df.head()


,flag_binding_arbitration,flag_limitation_liability,flag_governing_law,flag_venue_jurisdiction,flag_sole_discretion,flag_max_extent_law,flag_including_not_limited
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0


In [ ]:
# lengths align
assert len(val_txt)==len(VAL_TOK)==len(VAL_CHAR)==len(VAL_CPT)==len(VAL_NTOK_BUCKET)==len(VAL_NCHR_BUCKET)==len(VAL_CPT_BUCKET)

# buckets roughly balanced
import pandas as pd
print(pd.Series(VAL_NTOK_BUCKET).value_counts().sort_index())
print(pd.Series(VAL_NCHR_BUCKET).value_counts().sort_index())
print(pd.Series(VAL_CPT_BUCKET).value_counts().sort_index())

# flag hit counts
flag_df.sum().sort_values(ascending=False)

# verify risky labels are valid
set(sorted(RISKY)) - set(label_names)

# spot-check a few rows where a flag hit
hits = flag_df.index[flag_df.any(axis=1)].tolist()[:5]
[hits, [val_txt[i][:140] for i in hits]]


n_tok_Q1    2516
n_tok_Q2    2530
n_tok_Q3    2465
n_tok_Q4    2489
Name: count, dtype: int64
n_char_Q1    2506
n_char_Q2    2496
n_char_Q3    2499
n_char_Q4    2499
Name: count, dtype: int64
chars_per_tok_Q1    2500
chars_per_tok_Q2    2500
chars_per_tok_Q3    2508
chars_per_tok_Q4    2492
Name: count, dtype: int64


[[2, 5, 7, 10, 12],
 ['The substantive laws of the State of Texas will govern the validity, construction and enforcement of this Agreement. Venue for any action ar',
  'Each of the Company and its Subsidiaries: (A) is and at all times has been in compliance with all statutes, rules, or regulations applicable',
  'All questions concerning the construction, validity, enforcement and interpretation of the Transaction Documents shall be governed by and co',
  'The Corporation shall, to the fullest extent permitted by applicable law as it presently exists or may hereafter be amended, indemnify and h',
  'The Transactions (a)\xa0do not require any consent or approval of, registration or filing with or any other action by any Governmental Authorit']]

In [ ]:
from transformers import AutoTokenizer
BASE_ENC = "roberta-large"
tok = AutoTokenizer.from_pretrained(BASE_ENC, use_fast=True)
MAX_LEN = 384

from collections import Counter
counts = Counter(ds["train"]["label"])
counts_arr = np.array([counts[i] for i in range(num_labels)], dtype=float)
median_c = np.median(counts_arr)
class_weights = median_c / np.maximum(counts_arr, 1.0)  # heavier weight for rare labels

def tokenize(batch):
    return tok(batch["text"], truncation=True, padding=False, max_length=MAX_LEN)

tok_ds = ds.map(tokenize, batched=True)
tok_ds = tok_ds.rename_column("label", "labels")
tok_ds = tok_ds.remove_columns(["text"])

from transformers import DataCollatorWithPadding
collator = DataCollatorWithPadding(tok, pad_to_multiple_of=8, return_tensors="pt")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
# Cell B — RoBERTa + LoRA (no k-bit), safe targets
import torch, torch.nn.functional as F, random, numpy as np
from collections import Counter
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(42)

dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_ENC,            # "roberta-large"
    num_labels=num_labels,
    torch_dtype=dtype
)
model.config.use_cache = False  # recommended off for training

lcfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
    target_modules=["query","key","value","intermediate.dense","output.dense"],
    modules_to_save=["classifier"]
)

model = get_peft_model(model, lcfg)

# Class weights (recompute from your tokenized train split)
counts = Counter(tok_ds["train"]["labels"])
counts_arr = np.array([counts[i] for i in range(num_labels)], dtype=float)
median_c = np.median(counts_arr)
class_weights = median_c / np.maximum(counts_arr, 1.0)

# Weighted + label-smoothed loss Trainer
from transformers import Trainer
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**{k:v for k,v in inputs.items() if k!="labels"})
        logits = outputs.logits
        cw = torch.tensor(class_weights, device=logits.device, dtype=logits.dtype)
        eps = 0.1  # label smoothing
        log_probs = torch.log_softmax(logits, dim=-1)
        nll = -log_probs.gather(dim=-1, index=labels.unsqueeze(1)).squeeze(1)
        smooth = -log_probs.mean(dim=-1)
        loss_vec = (1 - eps) * nll + eps * smooth
        loss = (loss_vec * cw[labels]).mean()
        return (loss, outputs) if return_outputs else loss

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable/1e6:.2f}M / {total/1e6:.2f}M ({100*trainable/total:.2f}%)")


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable params: 8.23M / 363.69M (2.26%)


In [ ]:
optim="adamw_torch_fused"   # good on A100
bf16=torch.cuda.is_available()


In [ ]:
import evaluate
import torch
from transformers import TrainingArguments

# Metrics
acc = evaluate.load("accuracy")
f1m = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1m.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

OUT_DIR_CLS = "/content/termsconditioned_encoder_lora"

args = TrainingArguments(
    output_dir=OUT_DIR_CLS,
    eval_strategy="epoch",          # evaluate at end of each epoch
    save_strategy="epoch",          # save at end of each epoch
    load_best_model_at_end=False,   # <-- IMPORTANT: disable best-model logic
    metric_for_best_model=None,     # <-- no metric to track for "best"
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    num_train_epochs=5,             # solid, not crazy, no need for 8
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    weight_decay=0.1,
    max_grad_norm=1.0,
    fp16=False,
    bf16=torch.cuda.is_available(),
    logging_steps=100,
    report_to="none",
    seed=42,
    dataloader_num_workers=2,
    optim="adamw_bnb_8bit",
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    data_collator=collator,
    compute_metrics=compute_metrics,
    tokenizer=tok,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    callbacks=[],
)

train_out = trainer.train()
print(train_out)

eval_out = trainer.evaluate(tok_ds["validation"])
print("Final validation metrics:", eval_out)


Epoch,Training Loss,Validation Loss
1,1.563500,No log
2,1.150700,No log
3,1.142900,No log
4,1.045300,No log
5,1.040600,No log


TrainOutput(global_step=9375, training_loss=1.4201513346354167, metrics={'train_runtime': 4533.9734, 'train_samples_per_second': 66.167, 'train_steps_per_second': 2.068, 'total_flos': 1.969475302470574e+17, 'train_loss': 1.4201513346354167, 'epoch': 5.0})


Final validation metrics: {'eval_runtime': 47.8555, 'eval_samples_per_second': 208.962, 'eval_steps_per_second': 13.06, 'epoch': 5.0}


In [ ]:
import torch, numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, classification_report

model.eval()

val_loader = DataLoader(
    tok_ds["validation"],
    batch_size=64,
    shuffle=False,
    collate_fn=collator,
)

all_logits = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        labels = batch["labels"]
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}
        outputs = model(**inputs)
        all_logits.append(outputs.logits.cpu())
        all_labels.append(labels.cpu())

logits_val = torch.cat(all_logits, dim=0).numpy()
y_true = torch.cat(all_labels, dim=0).numpy()
y_pred = logits_val.argmax(axis=1)

acc = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro")

print("Validation accuracy:", acc)
print("Validation macro F1:", macro_f1)
print()

# sklearn was complaining because it inferred fewer classes than
# the 100 target_names you passed. We fix that by explicitly
# giving it labels that match the names.
labels = list(range(len(id2name)))  # [0, 1, ..., 99]

print("Per class (head):")
report = classification_report(
    y_true,
    y_pred,
    labels=labels,
    target_names=[id2name[i] for i in labels],
    digits=3,
    zero_division=0,
)
print(report[:1500])


Validation accuracy: 0.8152
Validation macro F1: 0.7423831071335973

Per class (head):
                              precision    recall  f1-score   support

                 Adjustments      0.729     0.896     0.804        48
                  Agreements      0.282     0.500     0.361        22
                  Amendments      0.905     0.817     0.859       257
        Anti-Corruption Laws      0.944     0.654     0.773        26
             Applicable Laws      0.364     0.116     0.176        69
                   Approvals      0.571     0.769     0.656        26
                 Arbitration      0.905     0.985     0.944        68
                 Assignments      0.888     0.841     0.864       208
                     Assigns      0.000     0.000     0.000         3
                   Authority      0.451     0.667     0.538        69
              Authorizations      0.649     0.453     0.533       106
                 Base Salary      0.974     0.982     0.978       114
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/termsconditioned_checkpoints"

# Create a parent dir once
import os
os.makedirs(SAVE_DIR, exist_ok=True)

# Copy the whole training folder (includes all checkpoints & final model)
!cp -r "/content/termsconditioned_encoder_lora" "$SAVE_DIR/encoder_lora_v1"
tok.save_pretrained("/content/termsconditioned_tokenizer")
!cp -r "/content/termsconditioned_tokenizer" "$SAVE_DIR/tokenizer_v1"


Mounted at /content/drive


In [ ]:
import json, os

OUT_DIR_CLS = "/content/termsconditioned_encoder_lora"

model.save_pretrained(OUT_DIR_CLS, safe_serialization=True)
tok.save_pretrained(OUT_DIR_CLS)

meta = {
    "base_model": BASE_ENC,
    "num_labels": num_labels,
    "label_names": label_names,
    "risky_families": sorted(list(RISKY)),
    "max_length": MAX_LEN,
}

with open(os.path.join(OUT_DIR_CLS, "termsconditioned_meta.json"), "w") as f:
    json.dump(meta, f, indent=2)

os.listdir(OUT_DIR_CLS)


['vocab.json',
 'merges.txt',
 'checkpoint-9375',
 'tokenizer_config.json',
 'adapter_config.json',
 'adapter_model.safetensors',
 'README.md',
 'special_tokens_map.json',
 'tokenizer.json',
 'checkpoint-3750',
 'termsconditioned_meta.json',
 'checkpoint-1875',
 'checkpoint-5625',
 '.ipynb_checkpoints',
 'checkpoint-7500']

In [ ]:
import numpy as np

logits = logits_val  # reuse what you already computed
y_true = y_true      # from your manual eval cell

def softmax(x):
    x = x - x.max(axis=1, keepdims=True)
    ex = np.exp(x)
    return ex / ex.sum(axis=1, keepdims=True)

def ece(probs, y, n_bins=15):
    p = probs.max(1)
    c = (probs.argmax(1) == y).astype(float)
    edges = np.linspace(0, 1, n_bins+1)
    err = 0.0
    for i in range(n_bins):
        m = (p >= edges[i]) & (p < edges[i+1])
        if m.sum() == 0:
            continue
        err += abs(c[m].mean() - p[m].mean()) * m.mean()
    return float(err)

T_grid = np.round(
    np.concatenate([np.linspace(0.5, 0.95, 10), np.linspace(1.0, 3.0, 21)]),
    2
)

best_T, best_e = None, 1e9
for T in T_grid:
    P = softmax(logits / T)
    e = ece(P, y_true)
    if e < best_e:
        best_e, best_T = e, float(T)

rawP = softmax(logits)
calP = softmax(logits / best_T)

SUMMARY = {
    "val_acc_raw": float((rawP.argmax(1) == y_true).mean()),
    "val_acc_cal": float((calP.argmax(1) == y_true).mean()),
    "ECE_raw": float(ece(rawP, y_true)),
    "ECE_cal": float(ece(calP, y_true)),
    "best_T": best_T,
    "n_val": int(len(y_true)),
    "mode": "Encoder+LoRA 8-bit",
}
SUMMARY


{'val_acc_raw': 0.8152,
 'val_acc_cal': 0.8152,
 'ECE_raw': 0.1154585200600326,
 'ECE_cal': 0.0220565241061151,
 'best_T': 0.8,
 'n_val': 10000,
 'mode': 'Encoder+LoRA 8-bit'}

In [ ]:
import pandas as pd

RISKY_IDS = {i for i, name in id2name.items() if name in RISKY}

def sweep_policy(prob, y_true, tau_grid, risky_ids):
    y_true = np.asarray(y_true)
    conf = prob.max(axis=1)
    y_pred = prob.argmax(axis=1)
    true_risk = np.isin(y_true, list(risky_ids))
    pred_risk = np.isin(y_pred, list(risky_ids))
    rows = []
    for tau in tau_grid:
        keep = conf >= tau
        kept_rate = float(keep.mean())
        abstain_rate = 1.0 - kept_rate

        mask = true_risk & keep
        denom = max(mask.sum(), 1)
        fgr = float(((~pred_risk) & mask).sum() / denom)
        risky_recall_kept = float((pred_risk & mask).sum() / denom)

        rows.append({
            "tau": float(tau),
            "kept_rate": kept_rate,
            "abstain_rate": abstain_rate,
            "false_green_rate": fgr,
            "risky_recall_kept": risky_recall_kept,
        })
    return pd.DataFrame(rows)

tau_grid = np.linspace(0.20, 0.99, 80)
policy_sweep = sweep_policy(calP, y_true, tau_grid, RISKY_IDS)

CAP, MIN_KEEP = 0.05, 0.25   # at most 5 percent false green, at least 25 percent auto kept

ok = policy_sweep.query("false_green_rate <= @CAP and kept_rate >= @MIN_KEEP")

if len(ok):
    pick = ok.sort_values(
        ["abstain_rate", "risky_recall_kept", "tau"],
        ascending=[True, False, True]
    ).iloc[0]
else:
    pick = policy_sweep.sort_values(
        ["false_green_rate", "abstain_rate", "tau"]
    ).iloc[0]

tau_star = float(pick["tau"])

OPERATING_POINT = {
    "tau_star": tau_star,
    "cap": CAP,
    "kept_rate_at_tau": float(pick["kept_rate"]),
    "abstain_rate_at_tau": float(pick["abstain_rate"]),
    "false_green_rate_at_tau": float(pick["false_green_rate"]),
    "risky_recall_kept_at_tau": float(pick["risky_recall_kept"]),
}
OPERATING_POINT, policy_sweep.tail(6)


({'tau_star': 0.6799999999999999,
  'cap': 0.05,
  'kept_rate_at_tau': 0.7251,
  'abstain_rate_at_tau': 0.27490000000000003,
  'false_green_rate_at_tau': 0.04952076677316294,
  'risky_recall_kept_at_tau': 0.950479233226837},
      tau  kept_rate  abstain_rate  false_green_rate  risky_recall_kept
 74  0.94     0.4633        0.5367          0.019608           0.980392
 75  0.95     0.4370        0.5630          0.010101           0.989899
 76  0.96     0.3986        0.6014          0.010417           0.989583
 77  0.97     0.3103        0.6897          0.005848           0.994152
 78  0.98     0.1110        0.8890          0.000000           1.000000
 79  0.99     0.0000        1.0000          0.000000           0.000000)

In [ ]:
# True + predicted labels + calibrated confidence
val_true_name = [id2name[int(i)] for i in y_true]
conf = calP.max(axis=1)
y_pred = calP.argmax(axis=1)
pred_name = [id2name[int(i)] for i in y_pred]

keep = conf >= OPERATING_POINT["tau_star"]
true_risk = np.isin(y_true, list(RISKY_IDS))
pred_risk = np.isin(y_pred, list(RISKY_IDS))

base = pd.DataFrame({
    "true_id":  y_true,
    "true_name": val_true_name,
    "pred_id":  y_pred,
    "pred_name": pred_name,
    "conf": conf,
    "keep": keep.astype(int),
    "true_risk": true_risk.astype(int),
    "pred_risk": pred_risk.astype(int),
    "n_tok_bucket": VAL_NTOK_BUCKET,
    "n_char_bucket": VAL_NCHR_BUCKET,
    "chars_per_tok_bucket": VAL_CPT_BUCKET,
}).reset_index(drop=True)

# attach flag columns (binding arbitration, venue, etc.)
base = pd.concat([base, flag_df.reset_index(drop=True)], axis=1)

base.head()

,true_id,true_name,pred_id,pred_name,conf,keep,true_risk,pred_risk,n_tok_bucket,n_char_bucket,chars_per_tok_bucket,flag_binding_arbitration,flag_limitation_liability,flag_governing_law,flag_venue_jurisdiction,flag_sole_discretion,flag_max_extent_law,flag_including_not_limited
0,2,Amendments,2,Amendments,0.783284,1,1,1,n_tok_Q2,n_char_Q2,chars_per_tok_Q1,0,0,0,0,0,0,0
1,2,Amendments,38,Entire Agreements,0.728118,1,1,0,n_tok_Q2,n_char_Q2,chars_per_tok_Q4,0,0,0,0,0,0,0
2,47,Governing Laws,47,Governing Laws,0.340343,0,1,1,n_tok_Q1,n_char_Q1,chars_per_tok_Q2,0,0,0,1,0,0,0
3,48,Headings,48,Headings,0.975987,1,0,0,n_tok_Q1,n_char_Q1,chars_per_tok_Q2,0,0,0,0,0,0,0
4,47,Governing Laws,4,Applicable Laws,0.454525,0,1,0,n_tok_Q1,n_char_Q1,chars_per_tok_Q2,0,0,0,0,0,0,0


In [ ]:
def agg_slice(df):
    n = len(df)
    kept = int(df["keep"].sum())

    # risky + kept = the pool where false green-lights are even possible
    mask_risky_kept = (df["true_risk"] == 1) & (df["keep"] == 1)
    n_risky_kept = int(mask_risky_kept.sum())
    denom = max(n_risky_kept, 1)

    false_green = float(
        ((df["true_risk"] == 1) & (df["keep"] == 1) & (df["pred_risk"] == 0)).sum()
        / denom
    )

    return pd.Series({
        "n": n,
        "kept": kept,
        "kept_rate": kept / n if n else 0.0,
        "n_risky_kept": n_risky_kept,
        "false_green_rate": false_green,
        "avg_conf": float(df["conf"].mean()),
    })

# --- family slices ---
family_table = (
    base.groupby("true_name")
        .apply(agg_slice)
        .sort_values("n", ascending=False)
)
family_table.index = ["family::" + i for i in family_table.index]

# --- length / readability buckets ---
bucket_table = pd.concat([
    base.groupby("n_tok_bucket").apply(agg_slice),
    base.groupby("n_char_bucket").apply(agg_slice),
    base.groupby("chars_per_tok_bucket").apply(agg_slice),
]).sort_values("n", ascending=False)

# --- flag slices ---
flag_rows = []
for col in FLAGS.keys():
    g = base[base[col] == 1]
    if len(g) == 0:
        flag_rows.append([col, 0, 0, 0.0, 0, 0.0, 0.0])
        continue
    s = agg_slice(g)
    flag_rows.append([
        col,
        s["n"],
        s["kept"],
        s["kept_rate"],
        s["n_risky_kept"],
        s["false_green_rate"],
        s["avg_conf"],
    ])

flag_table = pd.DataFrame(
    flag_rows,
    columns=["slice","n","kept","kept_rate","n_risky_kept","false_green_rate","avg_conf"]
).set_index("slice")

# --- harm_score = expected number of false green-lights in slice ---
family_table["harm_score"] = family_table["n_risky_kept"] * family_table["false_green_rate"]
bucket_table["harm_score"] = bucket_table["n_risky_kept"] * bucket_table["false_green_rate"]
flag_table["harm_score"]   = flag_table["n_risky_kept"] * flag_table["false_green_rate"]

family_table.head(12), bucket_table.head(12), flag_table.sort_values("harm_score", ascending=False).head(12)


/tmp/ipython-input-3554232574.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(agg_slice)
/tmp/ipython-input-3554232574.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  base.groupby("n_tok_bucket").apply(agg_slice),
/tmp/ipython-input-3554232574.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas th

(                               n   kept  kept_rate  n_risky_kept  \
 family::Governing Laws     494.0    3.0   0.006073           3.0   
 family::Counterparts       429.0  419.0   0.976690           0.0   
 family::Notices            398.0  368.0   0.924623           0.0   
 family::Entire Agreements  389.0  271.0   0.696658           0.0   
 family::Severability       332.0  318.0   0.957831           0.0   
 family::Survival           260.0  215.0   0.826923           0.0   
 family::Amendments         257.0  172.0   0.669261         172.0   
 family::Taxes              214.0  160.0   0.747664           0.0   
 family::Assignments        208.0  166.0   0.798077           0.0   
 family::Insurances         204.0  196.0   0.960784           0.0   
 family::Terminations       193.0  164.0   0.849741           0.0   
 family::Payments           175.0  144.0   0.822857           0.0   
 
                            false_green_rate  avg_conf  harm_score  
 family::Governing Laws         

In [ ]:
family_table.to_csv("/content/family_table.csv")
bucket_table.to_csv("/content/bucket_table.csv")
flag_table.to_csv("/content/flag_table.csv")
policy_sweep.to_csv("/content/policy_sweep.csv", index=False)


In [ ]:
import torch, re
import numpy as np
import pandas as pd

# Use the calibrated temperature and global tau
T_STAR = float(SUMMARY["best_T"])
TAU_STAR = float(OPERATING_POINT["tau_star"])

def flag_phrases(text):
    """
    Return a list of flag names (like 'flag_venue_jurisdiction')
    that match the regex patterns you defined in FLAGS.
    """
    low = text.lower()
    hits = []
    for name, pattern in FLAGS.items():
        if re.search(pattern, low):
            hits.append(name)
    return hits

def paragraph_probs(texts, batch_size=32):
    """
    Calibrated class probabilities for a list of raw paragraphs.
    Uses the same tokenizer, model, MAX_LEN, and T_STAR.
    """
    all_probs = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            chunk = texts[i:i+batch_size]
            enc = tok(
                chunk,
                truncation=True,
                padding=True,
                max_length=MAX_LEN,
                return_tensors="pt",
            ).to(model.device)
            out = model(**enc)
            logits = out.logits
            probs = torch.softmax(logits / T_STAR, dim=-1)
            all_probs.append(probs.cpu().numpy())
    return np.vstack(all_probs)

def triage_paragraph(text):
    """
    Core governance decision rule for ONE paragraph.
    Uses:
      - calibrated probs
      - global tau
      - risk family set
      - phrase flags
    Returns a Risk Card dict.
    """
    probs = paragraph_probs([text])[0]
    max_conf = float(probs.max())
    pred_id = int(probs.argmax())
    pred_family = id2name[pred_id]
    risky_family = pred_family in RISKY

    flags = flag_phrases(text)

    if max_conf < TAU_STAR:
        action = "Needs review"
    else:
        if risky_family:
            action = "Flag as risky"
        else:
            action = "Green-light"

    # Simple rationale span:
    # 1) if we find any phrase flag, surface the first flag label
    # 2) otherwise, show the first ~140 characters as a preview
    if flags:
        rationale_span = flags[0].replace("flag_", "").replace("_", " ")
    else:
        rationale_span = text[:140]

    return {
        "clause_family": pred_family,
        "confidence": round(max_conf, 3),
        "risky_family": risky_family,
        "flags": flags,
        "action": action,
        "rationale_span": rationale_span,
    }

def batch_triage(texts):
    """
    Vectorized version that returns a DataFrame of Risk Cards
    for a list of paragraphs.
    """
    probs = paragraph_probs(texts)
    max_conf = probs.max(axis=1)
    pred_ids = probs.argmax(axis=1)

    records = []
    for i, (t, p_id, c) in enumerate(zip(texts, pred_ids, max_conf)):
        family = id2name[int(p_id)]
        risky = family in RISKY
        flags = flag_phrases(t)
        if c < TAU_STAR:
            action = "Needs review"
        else:
            if risky:
                action = "Flag as risky"
            else:
                action = "Green-light"

        if flags:
            rationale_span = flags[0].replace("flag_", "").replace("_", " ")
        else:
            rationale_span = t[:140]

        records.append({
            "idx": i,
            "text": t,
            "clause_family": family,
            "confidence": float(round(c, 3)),
            "risky_family": risky,
            "flags": ",".join(flags),
            "action": action,
            "tau_used": TAU_STAR,
            "T_star": T_STAR,
        })

    return pd.DataFrame(records)


In [ ]:
# Use a few validation paragraphs as a sanity check
sample_idx = [0, 1, 2, 3, 4]
sample_texts = [val_txt[i] for i in sample_idx]
cards = batch_triage(sample_texts)

for i, row in cards.iterrows():
    print(f"=== Example {int(row['idx'])} ===")
    print("Text:", row["text"][:200].replace("\n"," ") + ("..." if len(row["text"])>200 else ""))
    print("Predicted family :", row["clause_family"])
    print("Risky family?     :", row["risky_family"])
    print("Confidence        :", row["confidence"])
    print("Flags             :", row["flags"])
    print("Action            :", row["action"])
    print("----")


=== Example 0 ===
Text: This Agreement may be amended from time to time by the Committee in its discretion; provided , however , that this Agreement may not be modified in a manner that would have an adverse effect on the Aw...
Predicted family : Amendments
Risky family?     : True
Confidence        : 0.7770000100135803
Flags             : 
Action            : Flag as risky
----
=== Example 1 ===
Text: This Agreement embodies the entire agreement between the parties and supersedes all prior agreements and understandings relating to the Property except for any confidentiality agreement binding on Pur...
Predicted family : Entire Agreements
Risky family?     : False
Confidence        : 0.7319999933242798
Flags             : 
Action            : Green-light
----
=== Example 2 ===
Text: The substantive laws of the State of Texas will govern the validity, construction and enforcement of this Agreement. Venue for any action arising under or in connection with this Agreement shall be in...
Pr

In [ ]:
val_cards = batch_triage(val_txt)

out_path = "/content/validation_risk_cards_tau_{:.2f}_T_{:.2f}.csv".format(TAU_STAR, T_STAR)
val_cards.to_csv(out_path, index=False)
print("Saved validation Risk Cards to:", out_path)
val_cards.head()


Saved validation Risk Cards to: /content/validation_risk_cards_tau_0.68_T_0.80.csv


,idx,text,clause_family,confidence,risky_family,flags,action,tau_used,T_star
0,0,This Agreement may be amended from time to tim...,Amendments,0.783,True,,Flag as risky,0.68,0.8
1,1,This Agreement embodies the entire agreement b...,Entire Agreements,0.724,False,,Green-light,0.68,0.8
2,2,The substantive laws of the State of Texas wil...,Governing Laws,0.340,True,flag_venue_jurisdiction,Needs review,0.68,0.8
3,3,Section headings in this Amendment are include...,Headings,0.977,False,,Green-light,0.68,0.8
4,4,This Amendment shall be interpreted and constr...,Applicable Laws,0.455,False,,Needs review,0.68,0.8


In [ ]:
def simulate_force_abstain_on_slice(base_df, slice_mask, name="(slice)"):
    """
    Force human review for any row where slice_mask is True:
      - set keep = 0 for those rows
      - recompute global kept_rate and false_green_rate on risky families
      - compute how many extra manual reviews this causes
    """
    sim = base_df.copy()
    original_kept = int(sim["keep"].sum())

    sim.loc[slice_mask, "keep"] = 0

    kept_rate = sim["keep"].mean()

    mask_risky_kept = (sim["true_risk"] == 1) & (sim["keep"] == 1)
    n_risky_kept = int(mask_risky_kept.sum())
    false_greens = int(
        ((sim["true_risk"] == 1) & (sim["keep"] == 1) & (sim["pred_risk"] == 0)).sum()
    )
    false_green_rate = false_greens / n_risky_kept if n_risky_kept > 0 else 0.0

    extra_reviews = original_kept - int(sim["keep"].sum())

    return {
        "slice_name": name,
        "kept_rate": kept_rate,
        "false_green_rate": false_green_rate,
        "n_risky_kept": n_risky_kept,
        "false_greens": false_greens,
        "extra_reviews": extra_reviews,
        "total_examples": len(sim),
    }

# Example 1: force review of all Governing Laws paragraphs
res_gov = simulate_force_abstain_on_slice(
    base,
    base["true_name"] == "Governing Laws",
    name="true_family == Governing Laws",
)

# Example 2: force review of all paragraphs with venue/jurisdiction flag
res_venue = simulate_force_abstain_on_slice(
    base,
    base["flag_venue_jurisdiction"] == 1,
    name="flag_venue_jurisdiction == 1",
)

res_gov, res_venue


({'slice_name': 'true_family == Governing Laws',
  'kept_rate': np.float64(0.7248),
  'false_green_rate': 0.04654895666131621,
  'n_risky_kept': 623,
  'false_greens': 29,
  'extra_reviews': 3,
  'total_examples': 10000},
 {'slice_name': 'flag_venue_jurisdiction == 1',
  'kept_rate': np.float64(0.6782),
  'false_green_rate': 0.042435424354243544,
  'n_risky_kept': 542,
  'false_greens': 23,
  'extra_reviews': 469,
  'total_examples': 10000})

For Governing Laws, using the global tau_star already satisfies our 5 percent false green cap. Tightening the threshold only for this family costs almost nothing (3 extra reviews) and keeps the false green rate under that cap.

In [ ]:
base["is_risky_true"] = base["true_risk"].astype(bool)
base["pred_risk_bool"] = base["pred_risk"].astype(bool)

# False green-light = true risky, predicted non-risky, and kept
base["is_false_green"] = (
    base["is_risky_true"] &
    (~base["pred_risk_bool"]) &
    (base["keep"] == 1)
)

risk_mask = base["is_risky_true"]

print("Total validation rows:", len(base))
print("Total risky rows:", int(risk_mask.sum()))
print("Total false green-lights:", int(base["is_false_green"].sum()))

Total validation rows: 10000
Total risky rows: 1576
Total false green-lights: 31


In [ ]:
# overall false green rate among risky clauses
fg_rate = base.loc[risk_mask, "is_false_green"].mean()
print("False green light rate among risky clauses:", fg_rate)

# by clause family (only families that ever appear as risky)
fam_table_risky = (
    base.loc[risk_mask]
        .groupby("true_name")
        .agg(
            n=("true_name", "size"),
            false_green_rate=("is_false_green", "mean"),
            avg_conf=("conf", "mean"),
        )
        .sort_values("false_green_rate", ascending=False)
)
fam_table_risky.head(15)


False green light rate among risky clauses: 0.01967005076142132


,n,false_green_rate,avg_conf
true_name,,,
Modifications,40,0.075000,0.703128
Waivers,155,0.064516,0.710214
Indemnity,47,0.063830,0.609176
Remedies,119,0.033613,0.630335
Jurisdictions,33,0.030303,0.439847
Submission To Jurisdiction,42,0.023810,0.669425
Amendments,257,0.019455,0.727026
Arbitration,68,0.014706,0.954328
Indemnifications,140,0.007143,0.605440


In [ ]:
# by length buckets (risky only)
len_table_risky = (
    base.loc[risk_mask]
        .groupby("n_tok_bucket")
        .agg(
            n=("n_tok_bucket", "size"),
            false_green_rate=("is_false_green", "mean"),
            avg_conf=("conf", "mean"),
        )
        .sort_values("false_green_rate", ascending=False)
)
print(len_table_risky)

# by phrase flags (risky only)
flag_long = []
for col in FLAGS.keys():
    m = base[col] == 1
    if m.sum() == 0:
        continue
    sub = base.loc[m & risk_mask]
    if len(sub) == 0:
        continue
    flag_long.append({
        "slice": col,
        "n": len(sub),
        "false_green_rate": sub["is_false_green"].mean(),
        "avg_conf": sub["conf"].mean(),
    })

flag_table_risky = pd.DataFrame(flag_long).sort_values("false_green_rate", ascending=False)
flag_table_risky


                n  false_green_rate  avg_conf
n_tok_bucket                                 
n_tok_Q3      342          0.035088  0.694511
n_tok_Q2      329          0.027356  0.674860
n_tok_Q4      433          0.013857  0.657787
n_tok_Q1      472          0.008475  0.627316


,slice,n,false_green_rate,avg_conf
6,flag_including_not_limited,39,0.051282,0.730254
4,flag_sole_discretion,29,0.034483,0.739864
0,flag_binding_arbitration,85,0.023529,0.866557
3,flag_venue_jurisdiction,380,0.021053,0.579209
2,flag_governing_law,198,0.015152,0.779949
1,flag_limitation_liability,1,0.000000,0.649097
5,flag_max_extent_law,7,0.000000,0.659495


In [ ]:
fam_table_risky.to_csv("/content/fam_table_risky_only.csv")
len_table_risky.to_csv("/content/len_table_risky_only.csv", index=True)
flag_table_risky.to_csv("/content/flag_table_risky_only.csv", index=False)

In [ ]:
sample_paragraphs = [
    "Any dispute, claim, or controversy arising out of or relating to the Services, these Terms, or the breach, termination, enforcement, interpretation, or validity thereof shall be resolved by binding arbitration administered by a recognized arbitration provider under its applicable rules. You understand and agree that by accepting these Terms you and the Company are each waiving the right to a trial before a judge or jury in a court of law.",
    "To the maximum extent permitted by applicable law, in no event shall the Company, its affiliates, directors, employees, or licensors be liable for any indirect, incidental, special, consequential, exemplary, or punitive damages, including loss of profits, lost revenue, loss of data, or business interruption, arising out of or in connection with your use of or inability to use the Services, whether based on warranty, contract, tort, or any other legal theory, even if the Company has been advised of the possibility of such damages.",
    "You agree to defend, indemnify, and hold harmless the Company and its affiliates, officers, directors, employees, and agents from and against any claims, liabilities, damages, losses, and expenses, including reasonable attorneys’ fees and costs, arising out of or in any way connected with (a) your access to or use of the Services, (b) your violation of these Terms, or (c) your violation of any third-party rights, including any intellectual property, privacy, or proprietary right.",
    "These Terms, together with any order forms or additional terms expressly incorporated by reference, constitute the entire agreement between you and the Company regarding the Services and supersede all prior or contemporaneous agreements, communications, and understandings. If any provision of these Terms is held to be invalid or unenforceable, that provision will be enforced to the maximum extent permissible, and the remaining provisions will remain in full force and effect.",
]

cards = batch_triage(sample_paragraphs)
cards


,idx,text,clause_family,confidence,risky_family,flags,action,tau_used,T_star
0,0,"Any dispute, claim, or controversy arising out...",Arbitration,0.953,True,flag_binding_arbitration,Flag as risky,0.68,0.8
1,1,To the maximum extent permitted by applicable ...,Remedies,0.244,True,flag_governing_law,Needs review,0.68,0.8
2,2,"You agree to defend, indemnify, and hold harml...",Indemnifications,0.586,True,,Needs review,0.68,0.8
3,3,"These Terms, together with any order forms or ...",Entire Agreements,0.366,False,,Needs review,0.68,0.8


In [41]:
# Cell 8 - Export everything needed to reuse the model and governance analysis

import os, shutil, json

EXPORT_DIR = "/content/termsconditioned_export"
os.makedirs(EXPORT_DIR, exist_ok=True)

print("Export dir:", EXPORT_DIR)

# 1) copy adapter + tokenizer artifacts from your training output dir
# OUT_DIR_CLS and BASE_ENC should already exist from earlier cells
print("Copying adapter and tokenizer files from:", OUT_DIR_CLS)

to_copy = [
    "adapter_config.json",
    "adapter_model.safetensors",
    "tokenizer.json",
    "tokenizer_config.json",
    "special_tokens_map.json",
    "vocab.json",
    "merges.txt",
]

for fname in to_copy:
    src = os.path.join(OUT_DIR_CLS, fname)
    dst = os.path.join(EXPORT_DIR, fname)
    if os.path.exists(src):
        shutil.copy(src, dst)
        print("  copied", fname)
    else:
        print("  missing", fname, "(skipped)")

# 2) save meta and metrics as JSON
meta = {
    "base_model": BASE_ENC,
    "num_labels": int(num_labels),
    "label_names": list(label_names),
    "risky_families": sorted(list(RISKY)),
    "summary": SUMMARY,
    "operating_point": OPERATING_POINT,
}

with open(os.path.join(EXPORT_DIR, "termsconditioned_meta.json"), "w") as f:
    json.dump(meta, f, indent=2)

print("Wrote termsconditioned_meta.json")

# 3) export slice tables
family_table.to_csv(os.path.join(EXPORT_DIR, "family_table.csv"))
bucket_table.to_csv(os.path.join(EXPORT_DIR, "bucket_table.csv"))
flag_table.to_csv(os.path.join(EXPORT_DIR, "flag_table.csv"))

print("Wrote family_table.csv, bucket_table.csv, flag_table.csv")


Export dir: /content/termsconditioned_export
Copying adapter and tokenizer files from: /content/termsconditioned_encoder_lora
  copied adapter_config.json
  copied adapter_model.safetensors
  copied tokenizer.json
  copied tokenizer_config.json
  copied special_tokens_map.json
  copied vocab.json
  copied merges.txt
Wrote termsconditioned_meta.json
Wrote family_table.csv, bucket_table.csv, flag_table.csv


In [43]:
# Cell 9 - Build an "audit_pack.md" summarizing metrics and worst slices

import numpy as np

lines = []

lines.append("# TermsConditioned – Validation Audit Pack\n")

# 1) global metrics
lines.append("## 1. Global classifier performance")
val_acc = float(SUMMARY.get("val_acc_cal", SUMMARY.get("val_acc_raw", 0.0)))
ece_cal = float(SUMMARY.get("ECE_cal", 0.0))
ece_raw = float(SUMMARY.get("ECE_raw", 0.0))

lines.append(f"- Validation accuracy (calibrated): {val_acc:.3f}")
lines.append(f"- Validation macro F1: {macro_f1:.3f}")
lines.append(f"- ECE before calibration: {ece_raw:.3f}")
lines.append(f"- ECE after calibration: {ece_cal:.3f}")
lines.append("")

# 2) operating point and global false green rate on risky clauses
lines.append("## 2. Operating point and policy cap")

cap = float(OPERATING_POINT["cap"])
tau_star = float(OPERATING_POINT["tau_star"])
keep_rate = float(OPERATING_POINT["kept_rate_at_tau"])
abstain_rate = float(OPERATING_POINT["abstain_rate_at_tau"])
fg_rate_tau = float(OPERATING_POINT["false_green_rate_at_tau"])
risky_recall_kept = float(OPERATING_POINT["risky_recall_kept_at_tau"])

# recompute global false green rate among risky clauses at this tau_star
mask_risky = base["true_risk"] == 1
is_false_green = (
    (base["true_risk"] == 1) &
    (base["keep"] == 1) &
    (base["pred_risk"] == 0)
)
global_fg_rate = float(is_false_green.sum() / max(mask_risky.sum(), 1))

lines.append(f"- Temperature T*: {SUMMARY['best_T']:.2f}")
lines.append(f"- Policy cap on false green-lights (risky clauses): {cap:.3f}")
lines.append(f"- Chosen threshold τ*: {tau_star:.3f}")
lines.append(f"- Keep rate at τ*: {keep_rate:.3f}")
lines.append(f"- Abstain rate at τ*: {abstain_rate:.3f}")
lines.append(f"- False green-light rate at τ* (within kept, risky): {fg_rate_tau:.3f}")
lines.append(f"- Global false green-light rate among all risky clauses: {global_fg_rate:.3f}")
lines.append(f"- Fraction of risky clauses that are kept at τ*: {risky_recall_kept:.3f}")
lines.append("")

# 3) worst clause families by harm
lines.append("## 3. Worst clause-family slices by harm_score")
top_family = (
    family_table.sort_values("harm_score", ascending=False)
                .head(8)
                .reset_index()
)

for _, row in top_family.iterrows():
    name = str(row["index"]).replace("family::", "")
    lines.append(
        f"- {name}: n={int(row['n'])}, kept_rate={row['kept_rate']:.3f}, "
        f"false_green_rate={row['false_green_rate']:.3f}, "
        f"harm_score={row['harm_score']:.3f}"
    )
lines.append("")

# 4) worst length / readability slices
lines.append("## 4. Length and readability slices")
top_bucket = (
    bucket_table.sort_values("harm_score", ascending=False)
                .head(8)
                .reset_index()
)

for _, row in top_bucket.iterrows():
    name = str(row["index"])
    lines.append(
        f"- {name}: n={int(row['n'])}, kept_rate={row['kept_rate']:.3f}, "
        f"false_green_rate={row['false_green_rate']:.3f}, "
        f"harm_score={row['harm_score']:.3f}"
    )
lines.append("")

# 5) worst phrase flags
lines.append("## 5. Phrase-flag slices (high-risk motifs)")
top_flags = (
    flag_table.sort_values("harm_score", ascending=False)
              .head(8)
              .reset_index()
)

for _, row in top_flags.iterrows():
    name = str(row["slice"])
    lines.append(
        f"- {name}: n={int(row['n'])}, kept_rate={row['kept_rate']:.3f}, "
        f"false_green_rate={row['false_green_rate']:.3f}, "
        f"harm_score={row['harm_score']:.3f}"
    )
lines.append("")

# 6) how to read the triage output
lines.append("## 6. Triage decision rule (for documentation)")
lines.append("- Classifier: roberta-large with LoRA adapters on LEDGAR clause families.")
lines.append("- Risk bucket: the following families are treated as risky:")
lines.append("  " + ", ".join(sorted(RISKY)))
lines.append("- Decision per paragraph:")
lines.append(f"  - compute calibrated probs with T*={SUMMARY['best_T']:.2f}")
lines.append("  - find max confidence and predicted family")
lines.append(f"  - if max_conf < τ*={tau_star:.3f} then action = Needs review")
lines.append("  - else if predicted family is risky then action = Flag as risky")
lines.append("  - else action = Green-light")
lines.append("- Phrase flags (regex) are used only for explanations and slice analysis,")
lines.append("  not for the core class label. A paragraph can have many flags.")
lines.append("")

audit_text = "\n".join(lines)

audit_path = os.path.join(EXPORT_DIR, "audit_pack.md")
with open(audit_path, "w") as f:
    f.write(audit_text)

print("Wrote audit pack to:", audit_path)
print()
print(audit_text)


Wrote audit pack to: /content/termsconditioned_export/audit_pack.md

# TermsConditioned – Validation Audit Pack

## 1. Global classifier performance
- Validation accuracy (calibrated): 0.815
- Validation macro F1: 0.742
- ECE before calibration: 0.115
- ECE after calibration: 0.022

## 2. Operating point and policy cap
- Temperature T*: 0.80
- Policy cap on false green-lights (risky clauses): 0.050
- Chosen threshold τ*: 0.680
- Keep rate at τ*: 0.725
- Abstain rate at τ*: 0.275
- False green-light rate at τ* (within kept, risky): 0.050
- Global false green-light rate among all risky clauses: 0.020
- Fraction of risky clauses that are kept at τ*: 0.950

## 3. Worst clause-family slices by harm_score
- Waivers: n=155, kept_rate=0.568, false_green_rate=0.114, harm_score=10.000
- Amendments: n=257, kept_rate=0.669, false_green_rate=0.029, harm_score=5.000
- Remedies: n=119, kept_rate=0.412, false_green_rate=0.082, harm_score=4.000
- Indemnity: n=47, kept_rate=0.277, false_green_rate=0.231

In [44]:
# Cell: ToS-level triage helper (run after the audit pack and triage code)

import re
import pandas as pd

def split_paragraphs(doc_text, min_chars=60):
    """
    Split a long ToS-like document into paragraphs.
    Very simple: split on blank lines and drop very short chunks.
    """
    raw_parts = re.split(r"\n\s*\n", doc_text.strip())
    parts = []
    for p in raw_parts:
        t = p.strip()
        if len(t) >= min_chars:
            parts.append(t)
    return parts

def triage_tos(doc_text, top_k=15):
    """
    Run batch_triage over a full ToS text.
    Returns a DataFrame of risk cards sorted by severity.
    Severity order: Flag as risky > Needs review > Green-light,
    then by confidence (descending).
    """
    paragraphs = split_paragraphs(doc_text)
    if not paragraphs:
        return pd.DataFrame(columns=[
            "para_idx","clause_family","risky_family",
            "confidence","flags","action","text"
        ])

    cards = batch_triage(paragraphs)

    # add a severity score for sorting
    severity_score = []
    for _, row in cards.iterrows():
        if row["action"] == "Flag as risky":
            sev = 2
        elif row["action"] == "Needs review":
            sev = 1
        else:
            sev = 0
        severity_score.append(sev)

    cards = cards.copy()
    cards["severity"] = severity_score
    cards["para_idx"] = cards["idx"]  # clearer name for external use

    # keep text in the table for now (first 200 chars)
    cards["text"] = cards["text"].apply(
        lambda t: t[:200].replace("\n", " ") + ("..." if len(t) > 200 else "")
    )

    cards_sorted = cards.sort_values(
        ["severity", "confidence"], ascending=[False, False]
    )

    if top_k is not None:
        cards_sorted = cards_sorted.head(top_k)

    return cards_sorted[[
        "para_idx","clause_family","risky_family",
        "confidence","flags","action","text"
    ]]

# quick sanity check on a synthetic mini-ToS
demo_tos = "\n\n".join([
    "Any dispute, claim, or controversy arising out of or relating to the Services shall be resolved by binding arbitration. "
    "You and the Company waive the right to a trial before a judge or jury.",
    "These Terms and any dispute arising from them shall be governed by the laws of the State of Washington.",
    "The failure of the Company to enforce any right shall not be deemed a waiver of such right.",
])

demo_cards = triage_tos(demo_tos, top_k=10)
demo_cards


,para_idx,clause_family,risky_family,confidence,flags,action,text
0,0,Arbitration,True,0.936,flag_binding_arbitration,Flag as risky,"Any dispute, claim, or controversy arising out..."
2,2,No Waivers,False,0.575,,Needs review,The failure of the Company to enforce any righ...
1,1,Governing Laws,True,0.548,,Needs review,These Terms and any dispute arising from them ...
